# K-Nearest Neighbors - Clasificación

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import tree
import numpy as np
import pandas as pd
import os

df = pd.read_csv(os.getcwd() + "\\data\\2008_small.csv",nrows=100000)

Seleccionaremos 4 columnas que vamos a usar como predictoras!

In [ ]:
newdf = df[["AirTime","Distance","TaxiOut","ArrDelay"]].dropna()
cols = newdf[newdf.columns[newdf.columns != "ArrDelay"]]

In [ ]:
cols

Y nos generaremos una columna respuesta categórica...

In [ ]:
filtro = newdf["ArrDelay"] > 10 

In [ ]:
newdf["ArrDelay"][filtro] = "Delayed"
newdf["ArrDelay"][filtro== False] = "Not Delayed"

In [ ]:
newdf

Ajustamos el modelo

In [ ]:
nbrs_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
nbrs_3.fit(cols,newdf["ArrDelay"]) # Entrenar

In [ ]:
predicciones_3 = nbrs_3.predict(cols) # Le he pedido predicciones

In [ ]:
predicciones_3

In [ ]:
np.mean(predicciones_3 == newdf["ArrDelay"])

In [ ]:
newdf["ArrDelay"].value_counts()/len(newdf["ArrDelay"])

In [ ]:
np.mean(predicciones_3 == newdf["ArrDelay"]) # Precisión / Accuracy

Probamos con distintas k...

In [ ]:
nbrs_1 = KNeighborsClassifier(n_neighbors=2)
nbrs_1.fit(cols,newdf["ArrDelay"])
predicciones_1 = nbrs_1.predict(cols)
np.mean(predicciones_1 == newdf["ArrDelay"])

Comparemos los resultados con la predicción más simple que podemos hacer... la media.

In [ ]:
np.mean(newdf["ArrDelay"] == "Not Delayed")

### Visualización de resultados de clasificación

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(newdf["ArrDelay"],predicciones_1)
print(confusion)

In [ ]:
import seaborn as sns
sns.heatmap( confusion, annot=True,fmt='.6g',
            xticklabels= nbrs_1.classes_, 
            yticklabels=nbrs_1.classes_)

¿Y para convertirla a porcentaje?

In [ ]:
confusion_matrix / np.sum(confusion_matrix)

In [ ]:
sns.heatmap( np.round(confusion_matrix / np.sum(confusion_matrix) * 100,2), annot=True,fmt='.6g',
            xticklabels= ["Delayed","Not Delayed"], 
            yticklabels=["Delayed","Not Delayed"]
           )

#### Introducción a la selección de parámetros

A continuación, vamos a ver un ejemplo de como seleccionar los parámetros usando bucles. Más adelante podremos ver maneras alternativas, pero es importante tener control manual sobre estos procesos.

In [ ]:
newdf = df[["AirTime","Distance","TaxiOut","TaxiIn","ArrDelay"]].dropna()
filtro = newdf["ArrDelay"] > 10 

cols = newdf[newdf.columns[newdf.columns != "ArrDelay"]]

newdf["ArrDelay"][filtro] = "Delayed"
newdf["ArrDelay"][filtro== False] = "Not Delayed"

In [ ]:
listaparametros = list(range(1,16))
listaprecision = []

for i in range(0,len(listaparametros)):
    clasificador = KNeighborsClassifier(n_neighbors=listaparametros[i])
    clasificador.fit(cols,newdf["ArrDelay"])
    predicciones = clasificador.predict(cols)
    listaprecision.append(np.mean(predicciones == newdf["ArrDelay"]))
    
    print("k=",listaparametros[i],"Precision:",listaprecision[i])


### Ejercicios KNN

1. Ajusta un modelo de KNN para los datos "iris". Qué columnas usaremos para clasificar y cuál va a ser la respuesta?

**(Intenta hacerlo usando validación externa!)**

2. Muestra los datos gráficamente

3. Prueba si es más productivo hacerlo con los datos escalados con media 0 y desviación 1 o con los datos originales.

4. Te animas a generar un bucle que permita escoger qué "k" es más favorable para estos datos?

5. Crea una matriz de confusión con los datos

6. (Extra) Representar gráficamente (nube de puntos) cuáles son los valores mal clasificados.

# Árboles de Clasificación

https://scikit-learn.org/stable/modules/tree.html

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cols, newdf["ArrDelay"], test_size=.2)

In [ ]:
y_train

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
np.mean(predictions == y_test)

In [ ]:
y_train.value_counts() / len(y_train)

### validación interna de un árbol

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(cols,newdf["ArrDelay"])
predictions = clf.predict(cols)
np.mean(predictions == newdf["ArrDelay"])

Ajustes de parámetros manuales

In [ ]:
clf = tree.DecisionTreeClassifier(min_samples_split = 0.005,  
                                  #class_weight = "balanced",
                                  max_features=4)
clf.fit(X_train,y_train)

# Evaluación externa
predictions = clf.predict(X_test)
np.mean(predictions == y_test)

### Ejercicios Árboles

1. Ajusta un modelo de árbol para los datos "iris".

2. Muestra los datos gráficamente como consideres, y representa el árbol de decisiones

3. Te animas a generar un bucle que permita escoger qué combinaciones de parámetros son más favorables para estos datos?

4. Crea una matriz de confusión con los datos

5. Explora otras opciones de evaluación de los modelos

https://towardsdatascience.com/how-to-best-evaluate-a-classification-model-2edb12bcc587

6. Implementa alguna de las soluciones

© Netmind S.L.

Todos los derechos reservados. Este documento (v1.00) ha sido diseñado para el uso exclusivo del cliente que atiende a esta formación.

Ninguna parte de este documento puede ser reproducida, distribuida o transmitida en cualquier forma o por cualquier medio sin el permiso previo por escrito de Netmind.

In [ ]:
from sklearn.metrics import f1_score
f1_score(newdf["ArrDelay"].apply(lambda x: 1 if x == "Not Delayed" else 0),
         pd.DataFrame(predicciones_1)[0].apply(lambda x: 1 if x == "Not Delayed" else 0))